In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#dataset entero: https://drive.google.com/drive/folders/1eIVVZ7J5wS7GhupNNfJZy2SH5mrJvmfg?usp=share_link
#dataset ordenado: https://drive.google.com/drive/folders/1ytqcX5DzI0wjkhzSpPbK4Y0dESxsSAH6?usp=sharing
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Se recorta y cambia el tamaño aleatorio de la imagen a 224x224 píxeles
        transforms.RandomHorizontalFlip(),  # Se aplica una inversión horizontal aleatoria a la imagen
        transforms.ToTensor(),  # Se convierte la imagen a un tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Se normaliza la imagen con valores predefinidos de media y desviación estándar
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),  # Se cambia el tamaño de la imagen a 256x256 píxeles
        transforms.CenterCrop(224),  # Se recorta el centro de la imagen a un tamaño de 224x224 píxeles
        transforms.ToTensor(),  # Se convierte la imagen a un tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Se normaliza la imagen con valores predefinidos de media y desviación estándar
    ]),
}

data_dir = '/content/drive/MyDrive/images'  # Directorio que contiene las imágenes
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}  # Se crean conjuntos de datos de imagen utilizando la estructura de directorios y las transformaciones definidas anteriormente


dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}  # Se calcula el tamaño de los conjuntos de datos de entrenamiento y validación
class_names = image_datasets['train'].classes  # Se obtienen los nombres de las clases a partir del conjunto de datos de entrenamiento


In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['val']))
inputs = inputs.swapaxes(1,3)  # Se intercambian los ejes de las imágenes para que coincidan con el formato esperado para la visualización

plt.figure(figsize=(10,10))  # Se establece el tamaño de la figura
for imagenes in range(16):
    plt.subplot(5,4,imagenes+1)  # Se crea una subparcela en la figura para cada imagen en el lote
    plt.xticks([])  # Se eliminan las marcas del eje x
    plt.yticks([])  # Se eliminan las marcas del eje y
    plt.grid(False)  # Se desactiva la cuadrícula
    plt.imshow(np.resize(inputs[imagenes], (224,224,3)))  # Se visualiza la imagen redimensionada
    plt.xlabel(classes[imagenes])  # Se establece el nombre de la clase como etiqueta del eje x


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()  # Se registra el tiempo de inicio del entrenamiento

    best_model_wts = copy.deepcopy(model.state_dict())  # Se realiza una copia de los pesos iniciales del modelo
    best_acc = 0.0  # Se inicializa la mejor precisión obtenida durante el entrenamiento como 0.0

    for epoch in range(num_epochs):  # Se itera sobre el número de épocas especificado
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))  # Se imprime el número de época actual
        print('-' * 10)

        # Cada época tiene una fase de entrenamiento y una de validación
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Se establece el modelo en modo de entrenamiento
            else:
                model.eval()   # Se establece el modelo en modo de evaluación

            running_loss = 0.0  # Se inicializa la pérdida acumulada en la época actual
            running_corrects = 0  # Se inicializa el número de predicciones correctas en la época actual

            # Iterar sobre los datos.
            for inputs, labels in dataloaders[phase]:  # Se itera sobre los batch de datos según la fase actual
                inputs = inputs.to(device)  # Se mueven los datos de entrada a la GPU si está disponible
                labels = labels.to(device)  # Se mueven las etiquetas a la GPU si está disponible

                # Se ponen a cero los gradientes de los parámetros
                optimizer.zero_grad()

                # Paso de adelante
                # Se rastrea el historial de operaciones si solo estamos en fase de entrenamiento
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)  # Se obtienen las predicciones del modelo
                    _, preds = torch.max(outputs, 1)  # Se obtienen las etiquetas predichas
                    loss = criterion(outputs, labels)  # Se calcula la pérdida

                    # Paso de atrás + optimización solo si estamos en fase de entrenamiento
                    if phase == 'train':
                        loss.backward()  # Se calculan los gradientes
                        optimizer.step()  # Se actualizan los pesos del modelo

                # Estadísticas
                running_loss += loss.item() * inputs.size(0)  # Se acumula la pérdida
                running_corrects += torch.sum(preds == labels.data)  # Se acumulan las predicciones correctas

            if phase == 'train':
                scheduler.step()  # Se actualiza el scheduler de acuerdo con la política de ajuste de la tasa de aprendizaje

            epoch_loss = running_loss / dataset_sizes[phase]  # Se calcula la pérdida promedio en la época actual
            epoch_acc = running_corrects.double() / dataset_sizes[phase]  # Se calcula la precisión promedio en la época actual

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))  # Se imprime la pérdida y precisión de la época actual

            # Se realiza una copia profunda del modelo si la precisión en validación es mejor que la mejor precisión anterior
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc  # Se actualiza la mejor precisión
                best_model_wts = copy.deepcopy(model.state_dict())  # Se realiza una copia profunda de los pesos del modelo

    time_elapsed = time.time() - since  # Se calcula el tiempo total de entrenamiento
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))  # Se imprime el tiempo total de entrenamiento
    print('Best val Acc: {:4f}'.format(best_acc))  # Se imprime la mejor precisión obtenida en validación

    # Se cargan los mejores pesos del modelo
    model.load_state_dict(best_model_wts)
    return model  # Se devuelve el modelo entrenado con los mejores pesos obtenidos


In [ ]:
#Esta función visualize_model está diseñada para visualizar el rendimiento del modelo en el conjunto de datos de validación.

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))  # Se transpone la entrada para cambiar el formato de los ejes
    mean = np.array([0.485, 0.456, 0.406])  # Media de normalización
    std = np.array([0.229, 0.224, 0.225])  # Desviación estándar de normalización
    inp = std * inp + mean  # Se aplica la normalización inversa
    inp = np.clip(inp, 0, 1)  # Se ajustan los valores de los píxeles para estar dentro del rango [0, 1]
    plt.imshow(inp)  # Se muestra la imagen
    if title is not None:
        plt.title(title)  # Se establece el título de la imagen si se proporciona
    plt.pause(0.001)  # Se pausa un poco para que las gráficas se actualicen

def visualize_model(model, num_images=10):
    was_training = model.training  # Se guarda el estado de entrenamiento actual del modelo
    model.eval()  # Se establece el modelo en modo de evaluación
    images_so_far = 0  # Se inicializa el contador de imágenes mostradas
    fig = plt.figure(figsize=(10,10))  # Se crea una nueva figura de matplotlib
    with torch.no_grad():  # Se deshabilita el cálculo de gradientes
        for i, (inputs, labels) in enumerate(dataloaders['val']):  # Se itera sobre el conjunto de datos de validación
            inputs = inputs.to(device)  # Se mueven los datos de entrada a la GPU si está disponible
            labels = labels.to(device)  # Se mueven las etiquetas a la GPU si está disponible

            outputs = model(inputs)  # Se obtienen las predicciones del modelo
            _, preds = torch.max(outputs, 1)  # Se obtienen las etiquetas predichas

            for j in range(inputs.size()[0]):  # Se itera sobre las imágenes en el lote
                images_so_far += 1  # Se incrementa el contador de imágenes mostradas
                ax = plt.subplot(4, 4, images_so_far)  # Se crea una subparcela en la figura
                ax.axis('off')  # Se desactivan los ejes
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))  # Se establece el título de la subparcela con la etiqueta predicha
                imshow(inputs.cpu().data[j])  # Se muestra la imagen actual

                if images_so_far == num_images:  # Si se ha mostrado el número deseado de imágenes
                    model.train(mode=was_training)  # Se restaura el modo de entrenamiento original del modelo
                    return  # Se sale de la función
        model.train(mode=was_training)  # Se restaura el modo de entrenamiento original del modelo al finalizar la visualización


In [ ]:
#Podéis encontrar más modelos en: https://pytorch.org/vision/main/models.html
model_ft = models.resnet18(weights = None)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,  len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

In [ ]:
visualize_model(model_ft)